In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Administrator\\Desktop\\Kidney-Disease-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Administrator\\Desktop\\Kidney-Disease-Classification'

In [5]:
# !pip install dagshub


import mlflow
import dagshub
dagshub.init(repo_owner='Siddhant20020', repo_name='Kidney-Disease-Classification-DL-Project', mlflow=True)

mlflow.end_run()  
mlflow.set_experiment("Kidney-Disease-Classification-Eval-2")


import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)


Accessing as Siddhant20020

Initialized MLflow to track repo "Siddhant20020/Kidney-Disease-Classification-DL-Project"

Repository Siddhant20020/Kidney-Disease-Classification-DL-Project initialized!

2026/01/30 12:25:12 INFO mlflow.tracking.fluent: Experiment with name 'Kidney-Disease-Classification-Eval-2' does not exist. Creating a new experiment.


In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from CNN_Classifier.constants import *
from CNN_Classifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone",
            mlflow_uri="https://dagshub.com/Siddhant20020/Kidney-Disease-Classification-DL-Project.mlflow/",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
import numpy as np

In [12]:

from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    f1_score,
    precision_score,
    recall_score
)


class Evaluation:
    def __init__(self, config):
        self.config = config

    # ----------------------------
    # Validation Generator
    # ----------------------------
    def _valid_generator(self):
        datagenerator_kwargs = dict(
            preprocessing_function=preprocess_input,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
            shuffle=False
        )

        valid_datagenerator = ImageDataGenerator(**datagenerator_kwargs)

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            **dataflow_kwargs
        )

    # ----------------------------
    # Load Model
    # ----------------------------
    def load_model(self):
        self.model = tf.keras.models.load_model(
            self.config.path_of_model,
            compile=False  
        )

        # Recompile SAFELY for evaluation
        self.model.compile(
            optimizer="adam",
            loss="categorical_crossentropy",
            metrics=["accuracy"]
        )

    # ----------------------------
    # Evaluation
    # ----------------------------
    def evaluation(self):
        self.load_model()
        self._valid_generator()

        # ---- Keras evaluation ----
        self.score = self.model.evaluate(
            self.valid_generator,
            verbose=1
        )

        # ---- Predictions ----
        y_true = self.valid_generator.classes
        y_pred_prob = self.model.predict(self.valid_generator, verbose=1)
        y_pred = np.argmax(y_pred_prob, axis=1)

        # ---- Metrics (SKLEARN) ----
        self.f1 = f1_score(y_true, y_pred, average="weighted")
        self.precision = precision_score(y_true, y_pred, average="weighted")
        self.recall = recall_score(y_true, y_pred, average="weighted")

        self.report = classification_report(
            y_true,
            y_pred,
            target_names=list(self.valid_generator.class_indices.keys())
        )

        self.cm = confusion_matrix(y_true, y_pred)

        print("\nClassification Report:\n", self.report)
        print("Confusion Matrix:\n", self.cm)

    # ----------------------------
    # Save Scores
    # ----------------------------
    def save_score(self):
        scores = {
            "loss": float(self.score[0]),
            "accuracy": float(self.score[1]),
            "f1_score": float(self.f1),
            "precision": float(self.precision),
            "recall": float(self.recall)
        }

        save_json(path=Path("scores.json"), data=scores)

    # ----------------------------
    # MLflow Logging
    # ----------------------------
    def log_into_mlflow(self):
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(
            mlflow.get_tracking_uri()
        ).scheme

        with mlflow.start_run(run_name="ResNet50_4Class_Eval"):
            mlflow.log_params(self.config.all_params)

            mlflow.log_metrics({
                "loss": self.score[0],
                "accuracy": self.score[1],
                "f1_score": self.f1,
                "precision": self.precision,
                "recall": self.recall
            })

            if tracking_url_type_store != "file":
                mlflow.keras.log_model(
                    self.model,
                    "model",
                    registered_model_name="ResNet50_Kidney_4Class"
                )
            else:
                mlflow.keras.log_model(self.model, "model")

In [ ]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2026-01-30 12:25:25,965:INFO:yaml file: config\config.yaml loaded successfully]
[2026-01-30 12:25:25,970:INFO:yaml file: params.yaml loaded successfully]
[2026-01-30 12:25:25,972:INFO:created directory at: artifacts]
Found 3732 images belonging to 4 classes.
234/234 [==============================] - 730s 3s/step

Classification Report:
               precision    recall  f1-score   support

        Cyst       0.86      0.84      0.85      1112
      Normal       0.99      0.83      0.90      1523
       Stone       0.63      0.92      0.75       413
       Tumor       0.72      0.80      0.76       684

    accuracy                           0.84      3732
   macro avg       0.80      0.85      0.82      3732
weighted avg       0.86      0.84      0.84      3732

Confusion Matrix:
 [[ 938    1  170    3]
 [   5 1269   51  198]
 [  12   12  379   10]
 [ 136    0    0  548]]


2026/01/30 12:48:06 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2026-01-30 12:48:58,258:WARNING:Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 54). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\ADMINI~1\AppData\Local\Temp\tmpabci_6rl\model\data\model\assets
[2026-01-30 12:49:17,702:INFO:Assets written to: C:\Users\ADMINI~1\AppData\Local\Temp\tmpabci_6rl\model\data\model\assets]


c:\Users\Administrator\Desktop\Kidney-Disease-Classification\venv\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'ResNet50_Kidney_4Class' already exists. Creating a new version of this model...
2026/01/30 12:52:18 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ResNet50_Kidney_4Class, version 3
Created version '3' of model 'ResNet50_Kidney_4Class'.
